In [1]:
from pyproj import Transformer
import numpy as np
import pandas as pd
import iris
import glob
import sys
import os
import cartopy.crs as ccrs
import itertools
from scipy import spatial
import numpy.ma as ma
import tilemapbase
import iris.plot as iplt
from math import cos, radians
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import time
from pyproj import Proj, transform
import warnings
import gc
import pickle
from collections import OrderedDict
from netCDF4 import Dataset
from Identify_Events_Functions import *
import concurrent.futures  # For parallel processing

pd.set_option('display.float_format', '{:.3f}'.format)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Custom limited-size cache
class LimitedSizeDict(OrderedDict):
    def __init__(self, *args, max_size=100, **kwargs):
        self.max_size = max_size
        super().__init__(*args, **kwargs)

    def __setitem__(self, key, value):
        if len(self) >= self.max_size:
            self.popitem(last=False)
        OrderedDict.__setitem__(self, key, value)

# Initialize a limited-size cache
cubes_cache = {
    'unfiltered': LimitedSizeDict(max_size=10),
    'filtered_100': LimitedSizeDict(max_size=10),
    'filtered_300': LimitedSizeDict(max_size=10)
}

def concatenate_with_error_handling(cube_list):
    problematic_cube_index = None
    start = 0
    for i, cube in enumerate(cube_list):
        try:
            concatenated_cube = cube_list[start:i+1].concatenate_cube()
        except Exception as e:
            print(f"Error concatenating cube {i}: {str(e)}")
            problematic_cube_index = i
            start = i
    
    if 0 <= problematic_cube_index < len(cube_list):
        del cube_list[problematic_cube_index]
        print(f"Cube at index {problematic_cube_index} successfully removed from the CubeList.")
    else:
        print(f"Index {problematic_cube_index} is out of range for CubeList.")
    
    concatenated_cube = cube_list.concatenate_cube() 
    return concatenated_cube

def load_and_cache_cube(year, cache, filenames_pattern):
    if year in cache:
        print(f"Using cached data for year {year}")
        return cache[year]

    print(f"Loading data for year {year}")
    filenames = [filename for filename in glob.glob(filenames_pattern) if '.nc' in filename]

    if not filenames:
        raise FileNotFoundError(f"No files found for the year {year} with pattern {filenames_pattern}")

    cubes = iris.load(filenames)
    for cube in cubes:
        cube.rename(cubes[0].name())
    iris.util.equalise_attributes(cubes) 
    
    try:
        full_day_cube = cubes.concatenate_cube()
    except:
        print("Error handling concatenation")
        full_day_cube = concatenate_with_error_handling(cubes)
    
    cache[year] = full_day_cube
    return full_day_cube

def save_cube_to_disk(cube, filepath):
    with open(filepath, 'wb') as f:
        pickle.dump(cube, f)

def load_cube_from_disk(filepath):
    with open(filepath, 'rb') as f:
        return pickle.load(f)

def find_gauge_Tb0_and_location_in_grid(gauge_num, sample_cube):
    gauge1 = tbo_vals.iloc[gauge_num]
    Tb0 = int(gauge1['Critical_interarrival_time'])
    closest_point, idx_2d = find_position_obs(sample_cube, gauge1['Lat'], gauge1['Lon'], plot_radius=10, plot=False)
    return Tb0, idx_2d

def find_amax_indy_events_v2(df, duration, Tb0):
    rainfall_cores = find_rainfall_core(df, duration=duration, Tb0=Tb0)
    rainfall_events_expanded = []

    for rainfall_core in rainfall_cores:
        rainfall_core_after_search1 = search1(df, rainfall_core)
        rainfall_core_after_search2 = search2(df, rainfall_core_after_search1)
        rainfall_core_after_search3 = search3(df, rainfall_core_after_search2, Tb0=Tb0)
        if len(rainfall_core_after_search3[rainfall_core_after_search3['precipitation (mm/hr)'] > 0.1]) > 0:
            rainfall_events_expanded.append(rainfall_core_after_search3)
    
    return rainfall_events_expanded

# Get Tb0 values at each gauge
tbo_vals = pd.read_csv('/nfs/a319/gy17m2a/PhD/datadir/RainGauge/interarrival_thresholds_CDD_noMissing.txt')

# Dataset paths and patterns
datasets = {
    'unfiltered': '/nfs/a161/gy17m2a/PhD/datadir/NIMROD/30mins/OriginalFormat_1km/Unfiltered/{year}/*',
    'filtered_100': '/nfs/a161/gy17m2a/PhD/datadir/NIMROD/30mins/OriginalFormat_1km/Filtered_100/{year}/*',
    'filtered_300': '/nfs/a161/gy17m2a/PhD/datadir/NIMROD/30mins/OriginalFormat_1km/Filtered_300/{year}/*'
}

yr = 2009
print(f"Processing year {yr}")

# Function to process each gauge
def process_gauge(gauge_num):
    if gauge_num in [423, 444, 827, 888]:
        return  # Skip specified gauges

    print(f"Processing gauge {gauge_num}")
    
    try:
        # Check if files are missing for this year across all datasets
        missing_files = False
        for dataset_name in datasets.keys():
            base_dir = f"../../../ProcessedData/IndependentEvents/NIMROD/NIMROD_1km_{dataset_name}/{gauge_num}/WholeYear/"
            if not any(os.path.exists(f"{base_dir}/{duration}hrs_{yr}_v2_part0.csv") for duration in [0.5, 1, 2, 3, 6, 12, 24]):
                missing_files = True
                break
                
        if missing_files:
            # Ensure directories for unfiltered, filtered_100, and filtered_300 exist
            for dataset_name in datasets.keys():
                base_dir = f"../../../ProcessedData/IndependentEvents/NIMROD/NIMROD_1km_{dataset_name}/{gauge_num}/WholeYear/"
                if not os.path.isdir(base_dir):
                    os.makedirs(base_dir)

            # Read in a sample cube for finding the location of gauge in grid
            sample_cube = iris.load(f'/nfs/a161/gy17m2a/PhD/datadir/NIMROD/30mins/OriginalFormat_1km/Unfiltered/{yr}/metoffice-c-band-rain-radar_uk_{yr}0602_30mins.nc')[0][1,:,:]

            # Find the Tb0 and index of this gauge
            Tb0, idx_2d = find_gauge_Tb0_and_location_in_grid(gauge_num, sample_cube)

            for dataset_name, dataset_path_pattern in datasets.items():
                print(f"Processing dataset: {dataset_name}")

                # Load data for this year
                general_filename = dataset_path_pattern.format(year=yr)
                cache_filepath = f"/nfs/a319/gy17m2a/PhD/datadir/cache/nimrod/WholeYear/cube_{yr}.pkl"

                try:
                    if yr not in cubes_cache[dataset_name]:
                        if os.path.exists(cache_filepath):
                            cube = load_cube_from_disk(cache_filepath)
                            print("Loading cube from cache")
                        else:
                            cube = load_and_cache_cube(yr, cubes_cache[dataset_name], general_filename)
                            save_cube_to_disk(cube, cache_filepath)
                            print("Loading cube from folder")
                    else:
                        cube = cubes_cache[dataset_name][yr]
                except (EOFError, FileNotFoundError) as e:
                    print(f"Error loading cube for year {yr}: {e}")
                    continue

                # Extract data for the specified indices
                data = cube[:, idx_2d[0], idx_2d[1]].data

                # Create a DataFrame from the data
                df = pd.DataFrame({
                    'times': cube[:, idx_2d[0], idx_2d[1]].coord('time').units.num2date(cube.coord('time').points),
                    'precipitation (mm/hr)': data,
                    'precipitation (mm)': data / 2})

                #############################################
                # Fill in missing values
                #############################################
                df['times'] = pd.to_datetime([t.isoformat() for t in df['times']])

                # Determine the frequency
                freq = '30T'  # 30 minutes

                # Create a full date range
                full_time_range = pd.date_range(start=df['times'].min(), end=df['times'].max(), freq=freq)

                # Set 'time' as index and reindex to the full range
                df.set_index('times', inplace=True)
                df = df.reindex(full_time_range)

                # Reset index and rename it back to 'time'
                df.reset_index(inplace=True)
                df.rename(columns={'index': 'times'}, inplace=True)

                # Calculate time difference in minutes
                df['minutes_since_last'] = df['times'].diff().dt.total_seconds() / 60
                df['minutes_since_last'] = df['minutes_since_last'].fillna(0)

                # Loop through durations
                for duration in [0.5, 1, 2, 3, 6, 12, 24]:
                    base_dir = f"../../../ProcessedData/IndependentEvents/NIMROD/NIMROD_1km_{dataset_name}/{gauge_num}/WholeYear/"
                    if not os.path.exists(f"{base_dir}/{duration}hrs_{yr}_v2_part0.csv"):
                        print(f"Finding the AMAX for {duration}hr events for gauge {gauge_num} in year {yr} for {dataset_name}")

                        # Find events
                        events_v2 = find_amax_indy_events_v2(df, duration=duration, Tb0=Tb0)

                        # Save events to CSV
                        for num, event in enumerate(events_v2):
                            if len(event) > 1:
                                event.to_csv(f"{base_dir}/{duration}hrs_{yr}_v2_part{num}.csv")

        else:
            print(f"All files already exist for gauge {gauge_num} and year {yr}")

        # Clear the cache at the end of processing each year
        for cache in cubes_cache.values():
            cache.clear()

        # Collect garbage to free up memory
        gc.collect()
    except Exception as e:
        print(f"Error processing gauge {gauge_num}: {e}")

# # Execute in parallel
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     gauge_nums = list(range(900,905))
#     executor.map(process_gauge, gauge_nums)

# print("Processing complete")


# Define the batch size
batch_size = 5
total_gauges = 1263  # Total number of gauges

# Calculate number of batches
num_batches = (total_gauges + batch_size - 1) // batch_size

# Loop through each batch
for batch_idx in range(num_batches):
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, total_gauges)
    
    print(f"Processing gauges from {start_idx} to {end_idx}")

    # Create a list of gauge numbers for the current batch
    gauge_nums = list(range(start_idx, end_idx))

    # Execute in parallel using ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(process_gauge, gauge_nums)

print("Processing complete")

/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is d

Processing year 2009


In [ ]:
# Define the batch size
batch_size = 5
total_gauges = 1263  # Total number of gauges

# Calculate number of batches
num_batches = (total_gauges + batch_size - 1) // batch_size

# Loop through each batch
for batch_idx in range(num_batches):
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, total_gauges)
    
    print(f"Processing gauges from {start_idx} to {end_idx}")

    # Create a list of gauge numbers for the current batch
    gauge_nums = list(range(start_idx, end_idx))

    # Execute in parallel using ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(process_gauge, gauge_nums)

print("Processing complete")

Processing gauges from 0 to 5
Processing gauge 0
Processing gauge 1
Processing gauge 2
Processing gauge 3
Processing gauge 4
All files already exist for gauge 1 and year 2009
All files already exist for gauge 0 and year 2009All files already exist for gauge 3 and year 2009
All files already exist for gauge 4 and year 2009

All files already exist for gauge 2 and year 2009
Processing gauges from 5 to 10
Processing gauge 5Processing gauge 6
Processing gauge 7
Processing gauge 8

Processing gauge 9
All files already exist for gauge 6 and year 2009
All files already exist for gauge 7 and year 2009
All files already exist for gauge 9 and year 2009
All files already exist for gauge 8 and year 2009
All files already exist for gauge 5 and year 2009
Processing gauges from 10 to 15
Processing gauge 10
Processing gauge 11
Processing gauge 12Processing gauge 13

Processing gauge 14
All files already exist for gauge 10 and year 2009
All files already exist for gauge 11 and year 2009
All files alrea

All files already exist for gauge 106 and year 2009
All files already exist for gauge 108 and year 2009
All files already exist for gauge 107 and year 2009
All files already exist for gauge 109 and year 2009
Processing gauges from 110 to 115
Processing gauge 110Processing gauge 111

Processing gauge 112
Processing gauge 113
Processing gauge 114
All files already exist for gauge 111 and year 2009
All files already exist for gauge 112 and year 2009
All files already exist for gauge 110 and year 2009
All files already exist for gauge 113 and year 2009
All files already exist for gauge 114 and year 2009
Processing gauges from 115 to 120
Processing gauge 115
Processing gauge 116
Processing gauge 117
Processing gauge 118
Processing gauge 119
All files already exist for gauge 115 and year 2009
All files already exist for gauge 119 and year 2009All files already exist for gauge 116 and year 2009
All files already exist for gauge 117 and year 2009
All files already exist for gauge 118 and year 

All files already exist for gauge 212 and year 2009All files already exist for gauge 214 and year 2009

Processing gauges from 215 to 220
Processing gauge 215
Processing gauge 216Processing gauge 217
Processing gauge 218
Processing gauge 219

All files already exist for gauge 215 and year 2009
All files already exist for gauge 219 and year 2009
All files already exist for gauge 217 and year 2009
All files already exist for gauge 216 and year 2009
All files already exist for gauge 218 and year 2009
Processing gauges from 220 to 225
Processing gauge 220
Processing gauge 221
Processing gauge 222
Processing gauge 223
Processing gauge 224
All files already exist for gauge 220 and year 2009
All files already exist for gauge 224 and year 2009All files already exist for gauge 223 and year 2009All files already exist for gauge 222 and year 2009


All files already exist for gauge 221 and year 2009
Processing gauges from 225 to 230
Processing gauge 225
Processing gauge 226
Processing gauge 227
P

All files already exist for gauge 318 and year 2009All files already exist for gauge 315 and year 2009All files already exist for gauge 316 and year 2009

All files already exist for gauge 319 and year 2009

Processing gauges from 320 to 325
Processing gauge 320
Processing gauge 321
Processing gauge 322
Processing gauge 323
Processing gauge 324
All files already exist for gauge 321 and year 2009
All files already exist for gauge 324 and year 2009
All files already exist for gauge 320 and year 2009
All files already exist for gauge 323 and year 2009
All files already exist for gauge 322 and year 2009
Processing gauges from 325 to 330
Processing gauge 325Processing gauge 326

Processing gauge 327
Processing gauge 328
Processing gauge 329
All files already exist for gauge 325 and year 2009
All files already exist for gauge 328 and year 2009
All files already exist for gauge 327 and year 2009All files already exist for gauge 326 and year 2009
All files already exist for gauge 329 and year 

All files already exist for gauge 420 and year 2009All files already exist for gauge 424 and year 2009All files already exist for gauge 422 and year 2009


Processing gauges from 425 to 430
Processing gauge 425Processing gauge 426
Processing gauge 427

Processing gauge 428
Processing gauge 429
All files already exist for gauge 427 and year 2009
All files already exist for gauge 426 and year 2009All files already exist for gauge 428 and year 2009All files already exist for gauge 425 and year 2009All files already exist for gauge 429 and year 2009



Processing gauges from 430 to 435
Processing gauge 430
Processing gauge 431
Processing gauge 432
Processing gauge 433Processing gauge 434

All files already exist for gauge 431 and year 2009
All files already exist for gauge 430 and year 2009
All files already exist for gauge 433 and year 2009
All files already exist for gauge 432 and year 2009All files already exist for gauge 434 and year 2009

Processing gauges from 435 to 440
Processing g

...

Finding the AMAX for 3hr events for gauge 510 in year 2009 for unfiltered
Error processing gauge 512: can't start new thread
Finding the AMAX for 6hr events for gauge 510 in year 2009 for unfiltered
Finding the AMAX for 12hr events for gauge 510 in year 2009 for unfiltered
Finding the AMAX for 0.5hr events for gauge 513 in year 2009 for filtered_100
Finding the AMAX for 24hr events for gauge 510 in year 2009 for unfiltered
2 eventsFinding the AMAX for 1hr events for gauge 513 in year 2009 for filtered_100

Finding the AMAX for 2hr events for gauge 513 in year 2009 for filtered_100
Processing dataset: filtered_100
Loading cube from cache
Finding the AMAX for 3hr events for gauge 513 in year 2009 for filtered_100
Finding the AMAX for 6hr events for gauge 513 in year 2009 for filtered_100
Finding the AMAX for 12hr events for gauge 513 in year 2009 for filtered_100
2 events
Finding the AMAX for 24hr events for gauge 513 in year 2009 for filtered_100
2 events
Processing dataset: filtered_30

Loading cube from cache
Finding the AMAX for 1hr events for gauge 519 in year 2009 for unfiltered
Finding the AMAX for 2hr events for gauge 517 in year 2009 for unfiltered
Finding the AMAX for 24hr events for gauge 518 in year 2009 for filtered_300
Finding the AMAX for 2hr events for gauge 519 in year 2009 for unfiltered
Finding the AMAX for 3hr events for gauge 517 in year 2009 for unfiltered
Finding the AMAX for 3hr events for gauge 519 in year 2009 for unfiltered
Finding the AMAX for 6hr events for gauge 517 in year 2009 for unfilteredFinding the AMAX for 6hr events for gauge 519 in year 2009 for unfiltered

Finding the AMAX for 12hr events for gauge 519 in year 2009 for unfiltered
Finding the AMAX for 12hr events for gauge 517 in year 2009 for unfiltered
2 events
Finding the AMAX for 24hr events for gauge 517 in year 2009 for unfiltered
2 events
Processing dataset: filtered_100Finding the AMAX for 24hr events for gauge 519 in year 2009 for unfiltered

Loading cube from cache
2 even

Finding the AMAX for 24hr events for gauge 522 in year 2009 for filtered_100
Finding the AMAX for 6hr events for gauge 523 in year 2009 for filtered_100
Finding the AMAX for 2hr events for gauge 521 in year 2009 for filtered_100
2 events
Finding the AMAX for 3hr events for gauge 521 in year 2009 for filtered_100
Finding the AMAX for 12hr events for gauge 523 in year 2009 for filtered_100
Finding the AMAX for 6hr events for gauge 521 in year 2009 for filtered_100
Processing dataset: filtered_300
Finding the AMAX for 24hr events for gauge 523 in year 2009 for filtered_100
Finding the AMAX for 12hr events for gauge 521 in year 2009 for filtered_100
Loading cube from cache
Processing dataset: filtered_300
Loading cube from cache
Finding the AMAX for 24hr events for gauge 521 in year 2009 for filtered_100
2 events
Processing dataset: filtered_300
Loading cube from cache
Finding the AMAX for 0.5hr events for gauge 522 in year 2009 for filtered_300Finding the AMAX for 0.5hr events for gauge 5

Finding the AMAX for 6hr events for gauge 525 in year 2009 for filtered_100
Finding the AMAX for 3hr events for gauge 526 in year 2009 for filtered_300
Finding the AMAX for 12hr events for gauge 525 in year 2009 for filtered_100
Finding the AMAX for 6hr events for gauge 526 in year 2009 for filtered_300
Finding the AMAX for 24hr events for gauge 525 in year 2009 for filtered_100
2 events
Finding the AMAX for 12hr events for gauge 526 in year 2009 for filtered_300
Finding the AMAX for 24hr events for gauge 526 in year 2009 for filtered_300
Processing dataset: filtered_300
Loading cube from cache
2 events
Finding the AMAX for 0.5hr events for gauge 527 in year 2009 for filtered_300
Finding the AMAX for 0.5hr events for gauge 525 in year 2009 for filtered_300
Finding the AMAX for 1hr events for gauge 527 in year 2009 for filtered_300
Finding the AMAX for 1hr events for gauge 525 in year 2009 for filtered_300
Finding the AMAX for 2hr events for gauge 527 in year 2009 for filtered_300
Findi

Finding the AMAX for 6hr events for gauge 531 in year 2009 for filtered_300
Finding the AMAX for 24hr events for gauge 530 in year 2009 for filtered_300
Finding the AMAX for 12hr events for gauge 531 in year 2009 for filtered_300
Finding the AMAX for 24hr events for gauge 531 in year 2009 for filtered_300
2 events
Processing gauges from 535 to 540
Processing gauge 535
Processing gauge 536
Processing gauge 537
Processing gauge 538
Processing gauge 539
(940, 805)
Processing dataset: unfiltered
Loading cube from cache
(673, 584)
Processing dataset: unfiltered
Loading cube from cache
(1036, 888)
Processing dataset: unfiltered
Loading cube from cache
